In [1]:
import numpy as np
import pandas as pd
import os
import itertools
from ase import Atoms
from ase.io import read
from ase.geometry import distance,get_distances
from ase.visualize import view

In [76]:


class LiCoordination:
    def __init__(self, *args):
        self.atoms_list = [self.recorrect_pos(arg.copy()) for arg in args]
        print(self.atoms_list)

    def __call__(self, *args, **kwargs):
        if len(self.atoms_list)==2:
            atoms = self.two_coordination(self.atoms_list)
            return atoms
        elif len(self.atoms_list)==3:
            atoms = self.three_coordination(self.atoms_list)
            return atoms


    @staticmethod
    def rotation_matrix(v1, v2):
        """return a rotation matrix of 2 vectors"""
        v1 = v1 / np.linalg.norm(v1)  # 归一化第一个向量
        v2 = v2 / np.linalg.norm(v2)  # 归一化第二个向量
        cos_theta = np.dot(v1, v2)  # 两个向量的点积
        sin_theta = np.linalg.norm(np.cross(v1, v2))  # 两个向量的叉积的模长
        axis = np.cross(v1, v2)  # 旋转轴
        axis /= np.linalg.norm(axis)  # 归一化旋转轴
        # 使用罗德里格斯公式计算旋转矩阵
        rotation_matrix = np.array([
            [cos_theta + (1 - cos_theta) * axis[0]**2, (1 - cos_theta) * axis[0] * axis[1] - sin_theta * axis[2], (1 - cos_theta) * axis[0] * axis[2] + sin_theta * axis[1]],
            [(1 - cos_theta) * axis[1] * axis[0] + sin_theta * axis[2], cos_theta + (1 - cos_theta) * axis[1]**2, (1 - cos_theta) * axis[1] * axis[2] - sin_theta * axis[0]],
            [(1 - cos_theta) * axis[2] * axis[0] - sin_theta * axis[1], (1 - cos_theta) * axis[2] * axis[1] + sin_theta * axis[0], cos_theta + (1 - cos_theta) * axis[2]**2]
        ])
        return rotation_matrix

    @classmethod
    def recorrect_pos(cls,atoms):
        """:return atoms towards the right"""
        li_index = cls.get_li_index(atoms)
        atoms_ = atoms[[atom.index for atom in atoms if atom.symbol != 'Li']]

        # get the mass center of the electrolyte molecular
        center  = atoms_.get_center_of_mass()
        # get the vector from Li to the mass center
        vector_li2center = atoms[li_index].position - center
        vector_x = np.array([1,0,0])
        # 计算旋转矩阵

        matrix_ = cls.rotation_matrix(vector_li2center,vector_x)

        atoms_rotated = np.dot(atoms.positions,matrix_.T)
        atoms.set_positions(atoms_rotated)
        return atoms



    @staticmethod
    def get_li_index(atoms):
        """获取原子系统中锂原子的索引"""
        for index, symbol in enumerate(atoms.get_chemical_symbols()):
            if symbol == "Li":
                return index

    @staticmethod
    def check_overlap(atoms1, atoms2, center ,threshold=1.5):
        """检查两个原子系统之间是否存在重叠"""
        dist_matrix = get_distances(atoms1.positions, atoms2.positions)
        print(dist_matrix[-1].min())
        if dist_matrix[-1].min() < threshold:
            print("The distance between atoms is too small!"
                  "Trying to adjust the distance...")
        while dist_matrix[-1].min() < threshold:
            c1 = atoms1.get_center_of_mass()
            c2 = atoms2.get_center_of_mass()
            v1 = center - c1
            v2 = center - c2

            # 计算0.1长度的移动向量
            scaled_vector_1  =  0.1 * v1 / np.linalg.norm(v1)
            atoms1.positions += scaled_vector_1
            scaled_vector_2  =  0.1 * v2 / np.linalg.norm(v2)
            atoms1.positions += scaled_vector_2

            dist_matrix = get_distances(atoms1.positions, atoms2.positions)

        return atoms1,atoms2


    def rotate_atoms(self, atoms, rotation_matrix, rotation_center):
        """对原子系统进行旋转和平移操作"""
        # 平移分子使旋转中心位于原点
        atoms_centered = atoms.positions - rotation_center.position
        # 使用旋转矩阵旋转分子
        atoms_rotated = atoms_centered.dot(rotation_matrix.T)
        atoms.set_positions(atoms_rotated)
        return atoms



    def two_coordination(self,atoms_list):
        """计算两个原子系统之间的二协同"""
        atoms1 = atoms_list[0]
        atoms2 = atoms_list[1]

        li_index_1 = self.get_li_index(atoms1)
        li_index_2 = self.get_li_index(atoms2)

        rotation_center_1 = atoms1[li_index_1]

        # 定义绕z轴旋转180度的旋转矩阵
        rotation_z = np.array([
            [-1, 0, 0],
            [0, -1, 0],
            [0, 0, 1]
        ])
        # 定义绕x轴旋转120度的旋转矩阵
        rotation_x = np.array([
            [1, 0, 0],
            [0, np.cos(np.pi * 2 / 3), -np.sin(np.pi * 2 / 3)],
            [0, np.sin(np.pi * 2 / 3), np.cos(np.pi * 2 / 3)]
        ])

        rotation_matrix = rotation_x.T @ rotation_z.T

        # 对原子系统进行旋转和平移操作
        atoms1_rotated = self.rotate_atoms(atoms1, rotation_matrix, rotation_center_1)

        # 平移 atoms1 使其与 atoms2 重叠
        center = atoms2[li_index_2].position
        atoms1_rotated.positions += center - atoms1[li_index_1].position

        # get the electrolyte molecules without Li atom
        atoms1_noli = atoms1[[atom.index for atom in atoms1 if atom.symbol != 'Li']]
        atoms2_noli = atoms2[[atom.index for atom in atoms2 if atom.symbol != 'Li']]

        # 检查重叠
        atoms1_moved,atoms2_moved = self.check_overlap(atoms1_noli, atoms2_noli,center=center)

        # 合并原子系统
        atoms = atoms2_moved + atoms1_noli
        return atoms

    def three_coordination(self,atoms_list):
        """计算三个原子系统之间的三协同"""
        atoms1 = atoms_list[0]
        atoms2 = atoms_list[1]
        atoms3 = atoms_list[2]

        li_index_1 = self.get_li_index(atoms1)
        li_index_2 = self.get_li_index(atoms2)
        li_index_3 = self.get_li_index(atoms3)

        rotation_center_1 = atoms1[li_index_1]
        rotation_center_2 = atoms2[li_index_2]
        rotation_center_3 = atoms3[li_index_3]

        # 定义绕z轴旋转120度的旋转矩阵
        rotation_matrix_z120 = np.array([
            [np.cos(2*np.pi/3), -np.sin(2*np.pi/3), 0],
            [np.sin(2*np.pi/3), np.cos(2*np.pi/3), 0],
            [0, 0, 1]
        ])

        # 定义绕x轴旋转60度的旋转矩阵
        rotation_matrix_x60 = np.array([
            [1, 0, 0],
            [0, np.cos(np.pi/3), -np.sin(np.pi/3)],
            [0, np.sin(np.pi/3), np.cos(np.pi/3)]
        ])

        # 对原子系统进行旋转和平移操作
        atoms1_rotated = self.rotate_atoms(atoms1, rotation_matrix_x60, rotation_center_1)
        atoms2_rotated = self.rotate_atoms(atoms2, rotation_matrix_z120, rotation_center_2)
        atoms3_rotated = self.rotate_atoms(atoms3, rotation_matrix_z120, rotation_center_3)

        # 平移 atoms1 和 atoms2 使其与 atoms3 重叠,atoms3 的Li原子作为center
        center = atoms3[li_index_3].position
        atoms1_rotated.positions += center - atoms1[li_index_1].position
        atoms2_rotated.positions += center - atoms2[li_index_2].position

        atoms1_noli = atoms1_rotated[[atom.index for atom in atoms1_rotated if atom.symbol != 'Li']]
        atoms2_noli = atoms2_rotated[[atom.index for atom in atoms2_rotated if atom.symbol != 'Li']]
        atoms3_noli = atoms3[[atom.index for atom in atoms3 if atom.symbol != 'Li']]

        # 检查重叠

        atoms1_moved,atoms2_moved = self.check_overlap(atoms1_noli, atoms2_noli,center)
        atoms1_moved,atoms3_moved = self.check_overlap(atoms1_moved, atoms3_noli,center)
        atoms2_moved,atoms3_moved = self.check_overlap(atoms2_moved, atoms3_noli,center)

        # 合并原子系统
        atoms = atoms3 + atoms1_moved + atoms2_moved
        return atoms

    def four_coordination(self, atoms_list):
        """计算四个原子系统之间的四协同"""
        atoms1 = atoms_list[0]
        atoms2 = atoms_list[1]
        atoms3 = atoms_list[2]
        atoms4 = atoms_list[3]

        li_index_1 = self.get_li_index(atoms1)
        li_index_2 = self.get_li_index(atoms2)
        li_index_3 = self.get_li_index(atoms3)
        li_index_4 = self.get_li_index(atoms4)

        rotation_center_1 = atoms1[li_index_1]
        rotation_center_2 = atoms2[li_index_2]
        rotation_center_3 = atoms3[li_index_3]
        rotation_center_4 = atoms4[li_index_4]

        # 定义绕z轴旋转120度的旋转矩阵
        rotation_matrix_z120 = np.array([
            [np.cos(2*np.pi/3), -np.sin(2*np.pi/3), 0],
            [np.sin(2*np.pi/3), np.cos(2*np.pi/3), 0],
            [0, 0, 1]
        ])

        # 定义绕z轴旋转240度的旋转矩阵
        rotation_matrix_z240 = np.array([
            [np.cos(4*np.pi/3), -np.sin(4*np.pi/3), 0],
            [np.sin(4*np.pi/3), np.cos(4*np.pi/3), 0],
            [0, 0, 1]
        ])

        # 定义绕x轴旋转60度的旋转矩阵
        rotation_matrix_x60 = np.array([
            [1, 0, 0],
            [0, np.cos(np.pi/3), -np.sin(np.pi/3)],
            [0, np.sin(np.pi/3), np.cos(np.pi/3)]
        ])

        # 定义绕x轴旋转300度的旋转矩阵
        rotation_matrix_x300 = np.array([
            [1, 0, 0],
            [0, np.cos(5*np.pi/6), -np.sin(5*np.pi/6)],
            [0, np.sin(5*np.pi/6), np.cos(5*np.pi/6)]
        ])

        # 对原子系统进行旋转和平移操作
        atoms1_rotated = self.rotate_atoms(atoms1, rotation_matrix_x60, rotation_center_1)
        atoms2_rotated = self.rotate_atoms(atoms2, rotation_matrix_z240, rotation_center_2)
        atoms3_rotated = self.rotate_atoms(atoms3, rotation_matrix_x300, rotation_center_3)
        atoms4_rotated = self.rotate_atoms(atoms4, rotation_matrix_x60, rotation_center_4)

        # 平移 atoms1、atoms2 和 atoms3 使其与 atoms4 重叠
        atoms1_rotated.positions += atoms4[li_index_4].position - atoms1[li_index_1].position
        atoms2_rotated.positions += atoms4[li_index_4].position - atoms2[li_index_2].position
        atoms3_rotated.positions += atoms4[li_index_4].position - atoms3[li_index_3].position

        atoms1_noli = atoms1_rotated[[atom.index for atom in atoms1_rotated if atom.symbol != 'Li']]
        atoms2_noli = atoms2_rotated[[atom.index for atom in atoms2_rotated if atom.symbol != 'Li']]
        atoms3_noli = atoms3_rotated[[atom.index for atom in atoms3_rotated if atom.symbol != 'Li']]
        atoms4_noli = atoms4[[atom.index for atom in atoms4 if atom.symbol != 'Li']]

        # 检查重叠
        if (
            self.check_overlap(atoms1_noli, atoms2_noli) or
            self.check_overlap(atoms1_noli, atoms3_noli) or
            self.check_overlap(atoms1_noli, atoms4_noli) or
            self.check_overlap(atoms2_noli, atoms3_noli) or
            self.check_overlap(atoms2_noli, atoms4_noli) or
            self.check_overlap(atoms3_noli, atoms4_noli)
        ):
            raise ValueError("The distance between atoms is too small!")

        # 合并原子系统
        atoms = atoms4 + atoms1_noli + atoms2_noli + atoms3_noli
        return atoms

In [77]:
atoms1 = read('./1Li/2DMP.xsd')
atoms2 = read('./1Li/13DIOXANE.xsd')
atoms3 = read('./1Li/BA/BA.log')

In [78]:
Li2 = LiCoordination(atoms1,atoms2)

[Atoms(symbols='O2C5H12Li', pbc=False), Atoms(symbols='O2C4H8Li', pbc=False)]


In [79]:
view(Li2())

3.704219404650972


<Popen: returncode: None args: ['D:\\ProgramData\\Anaconda3\\envs\\matminer\...>

In [21]:
li_coordination = LiCoordination()
atoms = li_coordination.two_coordination(atoms1,atoms2)

In [67]:
atoms3.positions

array([[ 9.905770e-01, -3.652090e-01,  9.700000e-05],
       [ 2.401390e+00,  1.364432e+00, -2.300000e-05],
       [-1.423546e+00, -3.676810e-01,  1.200000e-05],
       [-2.714458e+00,  4.648000e-01,  1.870000e-04],
       [-1.922350e-01,  5.150820e-01,  2.770000e-04],
       [-3.972168e+00, -4.087290e-01, -7.300000e-05],
       [ 2.189214e+00,  1.389610e-01,  2.510000e-04],
       [ 3.276842e+00, -8.890600e-01, -3.050000e-04],
       [-1.398240e+00, -1.017714e+00,  8.801640e-01],
       [-1.398186e+00, -1.017261e+00, -8.804720e-01],
       [-2.726906e+00,  1.121312e+00, -8.778620e-01],
       [-2.726958e+00,  1.120864e+00,  8.785700e-01],
       [-1.335270e-01,  1.144114e+00, -8.904050e-01],
       [-1.335830e-01,  1.143668e+00,  8.912770e-01],
       [-4.008205e+00, -1.051455e+00, -8.840180e-01],
       [-4.008253e+00, -1.051911e+00,  8.835390e-01],
       [-4.873960e+00,  2.069410e-01,  6.100000e-05],
       [ 3.169638e+00, -1.531581e+00, -8.774930e-01],
       [ 3.172807e+00, -1.52

In [66]:
get_distances(atoms3.positions, atoms3.positions + [3,3,3] )[1].min()

1.7066613504260297

In [16]:
def recorrect_pos(atoms):
    atoms = atoms.copy()
    li_index = LiCoordination.get_li_index(atoms)
    print(li_index)
    atoms_ = atoms[[atom.index for atom in atoms if atom.symbol != 'Li']]

    # get the mass center of the electrolyte molecular
    center  = atoms_.get_center_of_mass()
    # get the vector from Li to the mass center
    vector_li2center = atoms[li_index].position - center
    vector_x = np.array([1,0,0])
    # 计算旋转矩阵

    matrix_ = rotation_matrix(vector_li2center,vector_x)

    atoms_rotated = np.dot(atoms.positions,matrix_.T)
    atoms.set_positions(atoms_rotated)
    return atoms

In [18]:
view(recorrect_pos(atoms1))

19


<Popen: returncode: None args: ['D:\\ProgramData\\Anaconda3\\envs\\matminer\...>

In [7]:

# 示例用法
if __name__ == "__main__":
    # 创建示例原子系统 atoms1, atoms2, atoms3, atoms4
    # ...

    li_coordination = LiCoordination()

    # 计算二协同
    result_atoms = li_coordination.two_coordination(atoms1, atoms2)

    # 计算三协同
    result_atoms = li_coordination.three_coordination(atoms1, atoms2, atoms3)

    # 计算四协同
    # result_atoms = li_coordination.four_coordination(atoms1, atoms2, atoms3, atoms4)


ValueError: The distance between atoms is too small!